**Import Libraries**

In [39]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

**Processing of Frame**

In [40]:
def canny(img):

  if img is None:
    capture.release()
    cv2.destroyAllWindows()
    exit()

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  kernelSize = 5
  blur = cv2.GaussianBlur(gray, (kernelSize,kernelSize), 0 )
  cannyImg = cv2.Canny(blur, 50, 150)
  return cannyImg

In [41]:
def regionOfInterest(img):
  height = img.shape[0]
  width = img.shape[1]
  #to create an array 'mask' of the same dimensions as img and filled with 0
  #i.e. create a blank image of same size as the img and fills it black
  mask = np.zeros_like(img)
  #create an 1*3 array 'triangle' where each element is a tuble with x and y coordinates
  triangle_vertices = np.array([[
      (200,height),
      (800,350),
      (1200,height),]], np.int32)
  cv2.fillPoly(mask, triangle_vertices, 255)
  maskedImg = cv2.bitwise_and(img, mask)
  return maskedImg


In [42]:
def houghLines(img):
  houghLines = cv2.HoughLinesP(img, 2,np.pi/180, 100, np.array([]), minLineLength = 40, maxLineGap = 5)
  return houghLines

In [43]:
# a function which overlays the lines on an iamge
def displayLinesAverage (img, lines):
  #create a blank array of same size and img ie create a black image of same size in input image
  lineImage  = np.zeros_like(img)
  #checks if lines are present
  if lines is not None:
    for line in lines:
      #goes through each linesegment in lines, each will be defined by 4 points
      for x1,y1,x2,y2 in line:
        cv2.line(img,(x1,y1),(x2,y2), (255,0,0),10)
  return img

In [44]:
def makePoints(img, lineSI):
  slope, intercept = lineSI
  #y1 will be the image height because it will be at bottom of the image, can assume y2 is about 3/5th of the height or y1
  height = img.shape[0]
  y1 = int(height)
  y2 = int(y1*3.0/5)
  x1 = int((y1 - intercept)/slope)
  x2 = int((y2 - intercept)/slope)
  return [[x1,y1,x2,y2]]

In [45]:
def averageSlopeIntercept(img,lines):
  leftFit = []
  rightFit = []
  for line in lines:
    for x1,y1,x2,y2 in line:
      fit = np.polyfit((x1,x2),(y1,y2),1)
      slope = fit[0]
      intercept = fit[1]
      if slope < 0:
        leftFit.append((slope,intercept))
      else:
        rightFit.append((slope,intercept))
    #to find avg slope and intercept of left lane line and right lane line
  leftFitAverage = np.average(leftFit, axis=0)
  rightFitAverage = np.average(rightFit, axis=0)
  leftLine = makePoints(img, leftFitAverage)
  rightLine = makePoints(img, rightFitAverage)
  averageLines = [leftLine, rightLine]
  return averageLines


**Reading and Writing of Video**

In [49]:
from google.colab.patches import cv2_imshow
from google.colab import files

# The video file has been provided, please download the video onto your local machine
# After running this cell, it will upload onto files in google colab

uploaded = files.upload()

for file_name, file_content in uploaded.items():
    print(f"Uploaded file: {file_name}")

capture = cv2.VideoCapture(f'/content/{file_name}')

frame_width = int(capture.get(3))
frame_height = int(capture.get(4))
fps = capture.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter('/content/output_video.mp4', fourcc, fps, (frame_width, frame_height))

while capture.isOpened():
    ret, frame = capture.read()

    if not ret:
        break

    # Processing of each frame
    cannyOutput = canny(frame)
    maskedOutput = regionOfInterest(cannyOutput)
    lines = houghLines(maskedOutput)
    averageLinesImage = averageSlopeIntercept(frame, lines)
    lineImage = displayLinesAverage(frame, averageLinesImage)

    output_video.write(lineImage)

    if cv2.waitKey(20) & 0xFF == ord('d'):
        break

# Release the capture and writer objects
capture.release()
output_video.release()

# Destroy any OpenCV windows
cv2.destroyAllWindows()

Saving test1 (3).mp4 to test1 (3).mp4
Uploaded file: test1 (3).mp4


In [50]:
#The final lane detection video has been downloaded. Once downloaded, you can use your preferred video player to play the video locally
from google.colab import files
files.download('/content/output_video.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>